In [53]:
import psycopg2
import requests
import json
import time
from psycopg2.extras import Json
import pandas as pd
import numpy as np

In [54]:
conn = psycopg2.connect("dbname=postgres user=postgres host=localhost port=5432 password=123456")

In [55]:
base_url = 'https://api.hh.ru/vacancies'
params = {
    "text": "NAME:Аналитик",
    # "search_field":[ "exprerience", "schedule"], 
    "exprerience":"noExperience",
    "schedule": 'remote',
    "period": 3,
    "per_page": 100
}

In [56]:
r = requests.get(base_url, params=params)

In [57]:
j = json.loads(r.content)

In [58]:
j['found']

942

In [59]:
j['items'][4]

{'id': '105383904',
 'premium': False,
 'name': 'Аналитик Excel',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2024-09-06T11:07:16+0300',
 'created_at': '2024-09-06T11:07:16+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=105383904',
 'show_logo_in_search': None,
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/105383904?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/105383904',
 'relations': [],
 'employer': {'id': '10037549',
  'name': 'OptiTeam',
  'url': 'https://api.hh.ru/employers/10037549',
  'alternate_url': 'https://hh.ru/employer/10037549',
  'logo_urls': {'original': 'https://img.hhcdn.ru/employer-logo-original/1138182.png',
   '240': 'https://

In [60]:
cur = conn.cursor()
conn.commit()
cur.execute('create table analyst_2 (id int, area text, employer text, name text, salary_from int, salary_to int, currency text);')
cur.close()

In [61]:
cur = conn.cursor()
conn.commit()
cur.execute('truncate analyst_2;')
cur.close()

In [62]:
vac = j['items'][1]

In [63]:
vac['schedule']

{'id': 'remote', 'name': 'Удаленная работа'}

In [64]:
conn.commit()

In [65]:
def insert_vacancy(vac):
    cur = conn.cursor()
    cur.execute("insert into analyst_2 values (%s, %s, %s, %s, %s, %s, %s)",
                (vac['id'],                      
                 (lambda area: area['name'] if area else None)(vac['area']),
                 (lambda employer: employer['name'] if employer else None)(vac['employer']),
                 vac['name'],
                 (lambda salary: salary['from'] if salary else None)(vac['salary']),
                 (lambda salary: salary['to'] if salary else None)(vac['salary']),
                 (lambda schedule: schedule['currency'] if schedule else None)(vac['salary'])))
    conn.commit()
    cur.close()

In [66]:
def save_vacancy(page=0):
    print('Downloading page', page)
    r = requests.get(base_url, params=dict(params, page=page))
    time.sleep(1)
    result = json.loads(r.content)
    for v in result['items']:
        insert_vacancy(v)
    if len(result['items']) == 0:
        return
    else:
        save_vacancy(page + 1)

In [67]:
save_vacancy()

In [66]:
# cur = conn.cursor()
# cur.execute("select id, area, name, salary_from, salary_to, currency from analyst_2;")
# vacs = cur.fetchall()

In [67]:
# Данные записаны в базу, чтобы получить новые очистите базу

In [68]:
#######################